In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import requests
import time, os

In [ ]:
def GetAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Chrome/103.0.0.0"}) 
    soup = bts(result.text, "html.parser")
    return soup

In [ ]:
page_num = ["https://www.hepsiburada.com/fantastik-elyaf-astarli-pvc-masa-ortusu-silinebilir-sivi-gecirmez-leke-tutmaz-yaprak-kabartma-desenli-gumus-gri-p-HBCV00001VFT4K-yorumlari?"]
def find_page_num():
    for i in range(2,72):
        page_num.append("https://www.hepsiburada.com/fantastik-elyaf-astarli-pvc-masa-ortusu-silinebilir-sivi-gecirmez-leke-tutmaz-yaprak-kabartma-desenli-gumus-gri-p-HBCV00001VFT4K-yorumlari?sayfa=" + str(i))
    return page_num
find_page_num()

In [ ]:
### SAYFALAR SIRALARKEN DİKKAT ETMEMİZ GEREKEN HUSUS DÖNGÜYE ALDIĞIMIZ STRİNGİN NASIL OLDUĞUDUR.
### HEPSİBURADADA 2 ŞEKİLDE YORUM GÖRÜNTÜLENİYOR BİZ İLKİNİ TERCİH EDECEĞİZ.
### SONUNDA MAGAZA UZANTISI VARSA ONUNLA DONGU YAPMAYIN HATA VERİYOR .

####  'https://www.hepsiburada.com/fantastik-elyaf-astar-desenli-gumus-gri-p-HBCV00001VFT4K-yorumlari?'  ++++  TRUE
####  'https://www.hepsiburada.com/fantastik-elyaf-astardesenli-mor-p-HBCV00002DBZGK-yorumlari?magaza=NELTEKS'  ---- FALSE

In [ ]:
new_page = page_num[0:27]
##72 SAYFAMIZ OLMASINA RAGMEN 27 SAYFA ÇEKMEMİZİN SEBEBİ SADECE 27 SAYFADA YORUM OLMASIDIR.

In [ ]:
### DÖNGÜLERİ BAŞLATTIĞINIZDA FARKEDECEKSİZ Kİ DF ARASINDA SAYI AÇISINDAN UYUMSUZLUK VAR.."DF=500 DF1=499 DF2=499 GİBİ"
### BUNUN SEBEBİ BAZI YORUMLARIN SADECE FOTOĞRAFLA YAPILMIŞ OLMASIDIR.MAXİMUM 1 VEYA 2 SAYFADA BÖYLE BİR DURUM VARDIR VE
### GENELDE YORUMLARIN OLDUGU İLK VE SON SAYFADADIR BU SAYFALARI PAGE LİSTENİZDEN ÇIKARIRSANIZ SORUN AŞILACAKTIR.
### ÖRNEKLERİNİ MEDİUM YAZIMDA GÖREBİLİRİSNİZ.

In [ ]:
##BURADA YORUMUMUZ İÇİNDEKİ RATİNG YANİ YILDIZ SAYISINI ELDE EDİYORUZ.ARDINDAN DATAFRAME HALİNE GETİRİYORUZ.

result = []
for i in new_page:

    html = getAndParseURL(i)
    star = html.find_all("div", {"class":"hermes-RatingPointer-module-UefD0t2XvgGWsKdLkNoX"})
    for item in star :
        item1 = str(item)
        item2 = item1.count('star')
        result.append(item2)                                   
     

    time.sleep(2)   
    
## YUKARIDA DONGU İLE TUM SAYFALARI ELDE ETTİKTEN SONRA BTS İLE ALDIGIMIZ KOD BLOGU İÇERİSİNDEKİ STRİNG 'STAR' SAYISI 
# KADAR YARUMLARIMIZDA RATİNG VAR . 'HEPSİBURADA.COM' İÇİN ÇALIŞMAKLA BİRLİKTE DİĞER SİTELER İÇİN BİRAZ DEĞİŞTİRİMESİ GEREKMETEDİR.


In [ ]:
columns = ["skor"]
df = pd.DataFrame(result,columns = columns)                   #####FİRST######
len(df)
df


In [ ]:
result1 = []
for i in new_page:

    html = getAndParseURL(i)
    review = html.find_all('span',{'itemprop':'description'})
    for item in review :   
        item1 = str(item)
        item2 = item1.strip('<span itemprop="description">').strip("</'")
        result1.append(item2)
              
time.sleep(5)        

result1   

##BURADAKİ DONGUDE YORUMLARIN KENDİSİNİ ÇEKİYORUZ ve DF HALİNE GETİRİYORUZ.

In [ ]:
columns = ["reviews"]
df1 = pd.DataFrame(result1,columns = columns)
len(df1)
df1

In [ ]:
liste = []

for i in new_page:

    html = getAndParseURL(i)
    review = html.find_all('span',{'class':'hermes-ReviewCard-module-xvC02W_0j3edI0itfXVA'})
    for item in review :
            item1 = str(item)
            item2 = item1.strip('<span class="hermes-ReviewCard-module-xvC02W_0j3edI0itfXVA">').strip("</'").split('cm ')
            liste.append(item2)
        

liste_color = liste[1::2]
liste_size = liste[::2]
        
        
time.sleep(5) 

#BURADA YORUMLARIN YAPILDIĞI ÜRÜNÜN EBAT VE RENKLERİNİ ÇEKİP AYRI AYRI DF HALİNE GETİRİYORUZ.

In [ ]:
columns = ["size"]
df2 = pd.DataFrame(liste_olcu,columns = columns)
df2

In [ ]:
columns = ["color"]
df3 = pd.DataFrame(liste_renk,columns = columns)
df3

In [ ]:
new_df=pd.concat([df,df1,df2,df3],axis = 1)  #BİR ARAYA GETİRİYORUZ VE YENİ DF OLUŞTURUYORUZ.

In [ ]:
new_df.to_csv('hepsidata1.csv',index = False) # DF'Yİ PCYE ÇEKİYORUZ.